**<h2>Computing Water Potential Energy Surface Using Behler and Parinello Symmetry Functions** 

Importing relevant packages from PyTorch

In [ ]:
"""
Created on Sat May 15 17:34:55 2021
@author: Katerina Karoni
"""
import torch                        # Torch is an open-source machine learning library, a scientific computing framework,
                                       #and a script language
import torch.nn as nn
import torch.nn.functional as F     # Convolution Functions
import torch.optim as optim         # Package implementing various optimization algorithms
import matplotlib.pyplot as plt
from torchvision import datasets, transforms  #The torchvision package consists of popular datasets, model 
                                              #architectures, and common image transformations for computer vision
from torch.utils.data import DataLoader, TensorDataset       #Data loading utility class
from torch import Tensor
import pandas as pd

import numpy as np
import random
import os
import ase
from ase import Atoms
from ase.io import read

import math


Loading the data (energies and geometries) for 1000 water molecule configurations in .xyz form

In [ ]:
energies_water = np.genfromtxt('./water/energies.txt')
print("Energies file has",np.shape(energies_water),"entries")
#geometry_data =  read('./water/structures.xyz',index=':')
#print("Geometry file has",np.shape(geometry_data),"entries")
#print(geometry_data[0])
#geometry_data = np.array(geometry_data)

In [ ]:
# see https://education.molssi.org/python-data-analysis/01-numpy-arrays/index.html
#https://stackoverflow.com/questions/23353585/got-1-columns-instead-of-error-in-numpy

file_location = os.path.join('water', 'structures.xyz')
xyz_file = np.genfromtxt(fname=file_location, skip_header=2, dtype='unicode',invalid_raise = False)
# where invalid_raise = False was used to skip all lines in the xyz file that only have one column
symbols = xyz_file[:,0]
coordinates_water = (xyz_file[:,1:-1])
coordinates_water = coordinates_water.astype(np.float)
atomic_numbers_water = (xyz_file[:,-1])


In [ ]:
atomic_numbers_water = atomic_numbers_water.astype(int)
atomic_numbers_water = np.reshape(atomic_numbers_water,(len(coordinates_water),1))
#atomic_numbers_water = torch.from_numpy(atomic_numbers_water)
print(type(atomic_numbers_water))
print(atomic_numbers_water)
print(np.shape(atomic_numbers_water))

In [ ]:
## Alternatively loading the data in .npy form
# # The data was downloaded from http://www.quantum-machine.org/datasets/ (Densities dataset--> water.zip)
# # The data includes energies, densities and structure for water molecules
# #For each dataset, structures are given in with positions in Bohr and the energies are given in kcal/mol 
# energy_data = np.load('./water_102/dft_energies.npy')
# print("Energies file has",np.shape(energy_data),"entries")
# geometry_data =  np.load('./water-2/water_102/structures.npy')
# print("Geometry file has",np.shape(geometry_data),"entries")
# print(type(energy_data))
# print(type(geometry_data))

In [ ]:
print(coordinates_water)
print(np.shape(coordinates_water))

**<h2>Cutoff Function** 
    $$f_c(R_{ij}) = 
    \begin{cases}
        0.5 \times \big[\cos\big(\frac{\pi R_{ij}}{R_c}\big)+1\big]  & \text{for } R_{ij} \leq R_c\\
        0  & \text{for } R_{ij} > R_c
    \end{cases}
    $$
    
In the Behler and Parinello paper the Cutoff radius $R_c$ was taken to be $6$  Ångströms, or 11.3384 Bohr radii. (Remember, 1 Ångström is $10^{-10}$m)

In [ ]:
def fc(R,Rc):
    if R <= Rc:
        fcutoff = 0.5 * (np.cos(np.pi*R/Rc)+1)
    else:
        fcutoff = 0
    return fcutoff

In [ ]:
# Plotting fc as a function of interatomic distance Rij

Rc  = 11.3384 # Bohr

Rij     = np.linspace(0,Rc)
fcutoff = np.zeros(np.size(Rij))

for i in range(np.size(Rij)):
    fcutoff[i] = fc(Rij[i],Rc)

plt.plot(Rij,fcutoff)
plt.title('Cut-off function vs Interatomic distance', fontsize=16)
plt.xlabel('Interatomic Distance $R_{ij}$', fontsize=16)
plt.ylabel('$f_c(R_{ij})$', fontsize=16)
plt.grid()

**<h2>Pairwise Distances**

$$Dp = \begin{bmatrix} R_{00} & R_{01} & R_{02} \\ R_{10} & R_{11} & R_{12} \\ R_{20} & R_{21} & R_{22} \end{bmatrix} = \begin{bmatrix} 0 & R_{01} & R_{02} \\ R_{01} & 0 & R_{12} \\ R_{02} & R_{12} & 0 \end{bmatrix} $$

In [ ]:
i = 0                                     # i-th molecule
N = 3                                     # N atoms per molecule
coord = coordinates_water[N*i:N*(i+1),:]  # Let's take the coordinates of the ith water molecule in our dataset and compute
                                          # pairwise distances between all of its 3 atom
    
def pairwise_distances(coord):                       # we pass in the coordinates of the 3 atoms in the water molecule
    N = len(coord)
    pairwise_dist_matrix = np.zeros((N,N))       # Initialise the matrix
    for i in range(0,N-1):
        for j in range(i+1,N):
            pairwise_dist_matrix[i][j] =  np.sqrt(sum( (coord[i,:] - coord[j,:])**2 ))
            pairwise_dist_matrix[j][i] = pairwise_dist_matrix[i][j]
    return pairwise_dist_matrix

Dp = pairwise_distances(coord)
print(Dp)



**<h2>From Cartesian to Generalised Coordinates**

**<h3>Radial Symmetry Functions**
    
<h3>$$G_i^1 = \sum_{j \neq i}^{\text{all}} e^{-\eta (R_{ij}-R_s)^2} f_c (R_{ij})$$

In [ ]:
heta = 0.1
Rs   = 0
N    = 3

def radial_BP_symm_func(Dp,N,heta,Rs):
    G_mu1 = np.zeros(len(Dp))           # since we are dealing with water molecules the dimension of G will be 3 (H,H,O)
    for i in range(N):                 # to avoid using an if statement (if i not equal j), break the sum in two
        for j in range(0,i):
            G_mu1[i] = G_mu1[i] + np.exp(-heta*(Dp[i][j]-Rs)**2)* fc(Dp[i][j],Rc) 
        for j in range(i+1,N):
            G_mu1[i] = G_mu1[i] +  np.exp(-heta*(Dp[i][j]-Rs)**2)* fc(Dp[i][j],Rc) 
    return G_mu1

Gmu1 = radial_BP_symm_func(Dp,N,heta,Rs)
print(Gmu1)

**<h3> Angular Symmetry Functions**

$$G_i^2 = 2^{1-\zeta} \sum_{j,k \neq i}^{\text{all}} (1+\lambda \cos \theta_{ijk})^\zeta \times e^{-\eta (R_{ij}^2+R_{ik}^2+R_{jk}^2 )} f_c (R_{ij})f_c (R_{ik})f_c (R_{jk})$$
    
with parameters $\lambda = +1, -1$, $\eta$ and $\zeta$

In [ ]:
lambdaa = 1     
zeta    = 0.2
heta    = 0.1

N = len(coord)

def angular_BP_symm_func(coord,Dp,N,heta,Rs,lambdaa,zeta):
    G_mu2 = np.zeros(len(Dp))           # since we are dealing with water molecules the dimension of G will be 3 (H,H,O)

    for i in range(N):                 
        for j in range(N):           
            for k in range(N):
                if j != i and k !=i:
                    R_vec_ij = coord[i,:] - coord[j,:]
                    R_vec_ik = coord[i,:] - coord[k,:]
                    cos_theta_ijk  = np.dot(R_vec_ij, R_vec_ik)/(Dp[i][j]*Dp[i][k])
                    G_mu2[i]   = G_mu2[i] + (  1 + lambdaa * cos_theta_ijk )**zeta  \
                                * np.exp( -heta * (Dp[i][j]**2 + Dp[i][k]**2 + Dp[j][k]**2) ) \
                                * fc(Dp[i][j],Rc) * fc(Dp[i][k],Rc) * fc(Dp[j][k],Rc)            
        G_mu2[i]   = 2**(1-zeta) * G_mu2[i] 
    return G_mu2

Gmu2 = angular_BP_symm_func(coord,Dp,N,heta,Rs,lambdaa,zeta)
print(Gmu2)

In [ ]:
np.cos(180)

**<h2>Rotation functions**

**<h3>Rotation Matrix**

In [ ]:
def random_rotation_matrix():
    theta = np.arccos(2*np.random.uniform(low = 0,high = 1)-1)
    phi = np.random.uniform(low = 0,high = 2*np.pi)
    u = np.array([np.sin(theta)*np.cos(phi),np.sin(theta)*np.sin(phi),np.cos(theta)])
    theta = np.random.uniform(low = 0,high = 2*np.pi)
    A = np.zeros((3,3))
    A[0][0] = np.cos(theta) + (u[0]**2)*(1-np.cos(theta))
    A[0][1] = u[0]*u[1]*(1-np.cos(theta)) - u[2]*np.sin(theta)
    A[0][2] = u[0]*u[2]*(1-np.cos(theta)) + u[1]*np.sin(theta)
    A[1][0] = u[1]*u[0]*(1-np.cos(theta)) + u[2]*np.sin(theta)
    A[1][1] = np.cos(theta) + (u[1]**2)*(1-np.cos(theta))
    A[1][2] = u[1]*u[2]*(1-np.cos(theta)) - u[0]*np.sin(theta)
    A[2][0] = u[2]*u[0]*(1-np.cos(theta)) - u[1]*np.sin(theta)
    A[2][1] = u[2]*u[1]*(1-np.cos(theta)) + u[0]*np.sin(theta)
    A[2][2] = np.cos(theta) + (u[2]**2)*(1-np.cos(theta))
    return A


Function to randomly rotate molecules in the data set

In [ ]:
def rotate_data(A,data):
    data = np.array(data)
    m = np.shape(data)[1]
    for i in range(m):
        data[:,i] = np.matmul(A,data[:,i])
    return data

**<h2>Training and Test Set**

In [ ]:
N_H2O                    = 3       # number of atoms per molecule
number_of_features_H2O   = 15       # number of features (symmetry functions) for each atom (we create one radial)
                                   # and one angular, but can create more by vaying the parameters η, λ, ζ, Rs etc.
use_atom_num_as_feat     = int(0)       # Use atomic number of each element as an extra feature for training?
data_size_H2O            = np.shape(energies_water)[0]

In [ ]:
def create_dataset(N,number_of_features,coordinates,energies,atomic_numbers, use_atom_num_as_feat,batches):
    

    # Randomly picking the parameters from within a range
    heta   = np.linspace(0.01, 4, num=number_of_features)
    random.shuffle(heta)

    Rs     = np.linspace(0, 1, num=number_of_features)
    random.shuffle(Rs)

    lambdaa = np.ones(number_of_features)
    random.shuffle(lambdaa)

    zeta    = np.linspace(0, 8, num=number_of_features)
    random.shuffle(zeta)


    
# #     if number_of_features == 6:

# or set hand-picked ones
    heta    = [0.01,  4.,    3.202, 1.606, 2.404, 0.808, 1, 2, 0.1 , 0.05,1.2, 0.1, 0.3, 0.5,0.01] 
    zeta    = [8.,  1.6, 3.2, 4.8, 6.4, 0. , 9.,  1.9, 4.2, 5.8, 6.4, 0.001, 0.3, 2, 0.6 ]
    Rs      = [0.8, 0.4, 0.2, 1.,  0.,  0.6,   0.8, 0.4, 0.2, 1.,  0.,  0.6,  1, 0.1, 1.2]
    lambdaa = [1., 1., 1., 1., 1., 1.,1., 1., 1., 1., 1., 1., 1,1,1]
        


    data_size            = np.shape(energies)[0]        # We have 1000 water molecule conformations
    test_set_size        = 100
    training_set_size    = data_size - test_set_size

    
    #Randomly rotate each molecule in the data
    rotated_molec_coord = np.zeros((np.shape(coordinates)))
    for i in range(data_size):
        coord = coordinates_water[N*i:N*(i+1),:]
        coord = coord - coord[2,:]                   # move oxygen to origin for each molecule before rotating
        coord = np.transpose(coord)
        A = random_rotation_matrix()
        rotated_molec_coord[N*i:N*(i+1),:] = np.transpose(rotate_data(A,coord))
    
    coordinates = np.vstack((coordinates,rotated_molec_coord))  # randomly rotate trainig and test set and compute features for them as well for later use
    
    
    G = np.zeros((len(coordinates), number_of_features))  # we have 3000x2 features (2 symm funcs for ech of the 3000 atoms in the dataset)

    for i in range(2*data_size):                # factor 2 because we copied the data and randomly translated it
        coord = coordinates[N*i:N*(i+1),:]
        Dp    = pairwise_distances(coord)
        for j in range(0,number_of_features,2):
            if j < number_of_features - 1:      # for j = number_of_features compute either 1 or 2 symmetry functions depending on whether number_of_features is odd or even
                G[N*i:N*(i+1),j]   = radial_BP_symm_func(Dp,N,heta[j],Rs[j])
                G[N*i:N*(i+1),j+1] = angular_BP_symm_func(coord,Dp,N,heta[j],Rs[j],lambdaa[j],zeta[j])
            else:                  
                G[N*i:N*(i+1),j]   = radial_BP_symm_func(Dp,N,heta[j],Rs[j])
                if number_of_features % 2 == 0:    #i.e. if number of features is even
                    G[N*i:N*(i+1),j+1] = angular_BP_symm_func(coord,Dp,N,heta[j],Rs[j],lambdaa[j],zeta[j])

    
    # Computing variance and mean on the training data only!
    G_train = G[:training_set_size,:]
    var  = np.std(G_train,axis=0)
    mean = np.mean(G_train,axis=0)
    print(mean)
    G_norm = np.zeros((len(coordinates), number_of_features))
    print(np.shape(G))
    # normalize all data (training and test), using training set mean and variance
    for i in range(np.shape(G)[0]):
        for j in range(np.shape(G)[1]):
            G_norm[i,j] = (G[i,j]-mean[j])/var[j]   
        
    print(type(G_norm))  
    
    if use_atom_num_as_feat == 1:
        G_norm = np.append(G_norm, atomic_numbers, axis=1)    # Adding atomic number as a feature
       
        
    data_set = np.vsplit(G_norm,data_size*2)       # Group every three atoms that constitute a molecule together
    #data_set = np.random.permutation(training_set)
    data_set = torch.FloatTensor(data_set)                    # Going from a (1000,3,2) list to a a (1000,3,2) tensor
   
    # print(data_set[0])
    # print(data_set[0][1][1])

    labels = energies          # turning energies into a (1000) tensor

    
# #     print(np.shape(labels))
# #     print(np.shape(data_set))
#     shuffler = np.random.permutation(len(labels))

#     data_set = data_set[shuffler]

#     labels = labels[shuffler]

# #     print(np.shape(labels))
# #     print(np.shape(data_set))
    
    

    # Computing variance and mean on the training data only!
    lab_train = labels[:training_set_size]
    var_lab  = np.std(lab_train,axis=0)
    mean_lab = np.mean(lab_train,axis=0)

    labels_norm = np.zeros((np.shape(labels)))
    # normalize all data (training and test), using training set mean and variance
    for i in range(np.shape(labels)[0]):
        labels_norm[i] = (labels[i]-mean_lab)/var_lab  
    
    
    labels_norm = torch.FloatTensor(labels_norm)      
    
    
    # Splitting the dataset into training and test set
    training_set         = data_set[:training_set_size]
    test_set             = data_set[training_set_size:data_size]
    training_set_rot     = data_set[data_size:data_size+training_set_size]
    test_set_rot         = data_set[data_size+training_set_size:]

    train_labels         = labels_norm[:training_set_size]
    train_labels         = torch.FloatTensor(train_labels)
    test_labels          = labels_norm[training_set_size:]
    test_labels          = torch.FloatTensor(test_labels)

    # Dataset
    dataset = TensorDataset(training_set, train_labels)
    #print(dataset[0])

    # Creating the batches
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batches, #25, #300,
                                           shuffle=True, num_workers=2, drop_last=False) # ?????

    print(np.shape(training_set))
    
    return [training_set, test_set, train_labels, test_labels, dataloader,var_lab,mean_lab, test_set_rot,labels_norm,training_set_rot]

In [ ]:
data_water = create_dataset(N_H2O, number_of_features_H2O ,coordinates_water,energies_water, \
                      atomic_numbers_water,use_atom_num_as_feat, 300)



In [ ]:
training_set = data_water[0]
print('Training set:')
print(type(training_set))
print(np.shape(training_set))

test_set     = data_water[1]
print('\n')
print('Test set:')
print(type(test_set))
print(np.shape(test_set))

train_labels = data_water[2]
print('\n')
print('Training labels:')
print(type(train_labels))
print(np.shape(train_labels))

test_labels  = data_water[3]
print('\n')
print('Test labels:')
print(type(test_labels))
print(np.shape(test_labels))

dataloader   = data_water[4]
print('\n')
print('data_waterloader:')
print(type(dataloader))
print(np.shape(dataloader))

var_lab = data_water[5]
print('\n')
print('Variance of labels:')
print(type(var_lab))
print(var_lab)

mean_lab = data_water[6]
print('\n')
print('Mean value of labels:')
print(type(mean_lab))
print(mean_lab)

test_set_rot = data_water[7]
print('\n')
print('Rotated test set:')
print(type(test_set_rot))
print(np.shape(test_set_rot))

labels_norm = data_water[8]
print('\n')
print('Normalised labels:')
print(type(labels_norm))
print(np.shape(labels_norm))

training_set_rot = data_water[9]

In [ ]:
print(training_set)

**<h2>Building Neural Network Architecture**

In [ ]:
a = int(math.sqrt(4))
print(a)

In [ ]:
class Subnets_H2O(nn.Module):
    def __init__(self,number_of_features):
        super(Subnets_H2O, self).__init__()
        num_hid_feat = 10#math.ceil(number_of_features/2)#int(math.sqrt(number_of_features))##
        self.fc1 = nn.Linear(number_of_features, num_hid_feat)        # where fc stands for fully connected 
        self.fc2 = nn.Linear(num_hid_feat, num_hid_feat)
        self.fc3 = nn.Linear(num_hid_feat, 1)


    def forward(self, x,train = True):
        x = torch.relu(self.fc1(x))           # Apply a tanh activation on fully connected layer 1 
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)                   # Using a linear function (identity function) for the subnet output layers
        return x

class BPNN_H2O(nn.Module):
    def __init__(self,number_of_features):
        super(BPNN_H2O, self).__init__()
        self.network1 = Subnets_H2O(number_of_features)
        self.network2 = Subnets_H2O(number_of_features)
        self.network3 = Subnets_H2O(number_of_features)
        
#        self.fc_out = nn.Linear(3, 1)      # should this be defined here, given that we are not trying to optimise
        
    def forward(self, x1, x2, x3,train = True):
        x1 = self.network1(x1)
        x2 = self.network2(x2)
        x3 = self.network3(x3)
        
        
        x = torch.cat((x1, x2, x3), 0) 
#        x = self.fc_out(x)
        x = torch.sum(x)                   #??????????????????????????? try average pooling?
        x = torch.reshape(x,[1])
        return x

    
# model = BPNN_H2O(3) #+1)                # +1 because he have added the atomic number as a feature
# x1, x2, x3 = training_set[0]
# print('x1',x1)
# print('x2',x2)
# print('x3',x3)


# output = model(x1, x2, x3,number_of_features_H2O)# +1)
# print('output')
# print(output*var_lab+mean_lab)


# print(model)

# print('Network1')

# print('layer 1')
# print('weights')
# print(model.network1.fc1.weight)
# print('biases')
# print(model.network1.fc1.bias)

# print('layer 2')
# print('weights')
# print(model.network1.fc2.weight)
# print('biases')
# print(model.network1.fc2.bias)

# print('layer 3')
# print('weights')
# print(model.network1.fc3.weight)
# print('biases')
# print(model.network1.fc3.bias)

# print('Network2')

# print('layer 1')
# print('weights')
# print(model.network2.fc1.weight)
# print('biases')
# print(model.network2.fc1.bias)

# print('layer 2')
# print('weights')
# print(model.network2.fc2.weight)
# print('biases')
# print(model.network2.fc2.bias)

# print('layer 3')
# print('weights')
# print(model.network2.fc3.weight)
# print('biases')
# print(model.network2.fc3.bias)

# print('Network3')

# print('layer 1')
# print('weights')
# print(model.network3.fc1.weight)
# print('biases')
# print(model.network3.fc1.bias)

# print('layer 2')
# print('weights')
# print(model.network3.fc2.weight)
# print('biases')
# print(model.network3.fc2.bias)


# print('layer 3')
# print('weights')
# print(model.network3.fc3.weight)
# print('biases')
# print(model.network3.fc3.bias)

In [ ]:
# x1, x2, x3 = training_set[0]
# print('x1',x1)
# print('x2',x2)
# print('x3',x3)


# output = model(x1, x2, x3,number_of_features_H2O+1)
# print('output')
# print(output*var_lab+mean_lab)

In [ ]:
## Manually compute neural Network output

# w11 = model.network1.fc1.weight
# b11 = model.network1.fc1.bias
# print(np.shape(w11))
# x1 = np.reshape(x1,(2,1))
# x1 = np.array(x1)
# print(np.shape(x1))

# w11 = w11.cpu().detach().numpy()
# b11 = b11.cpu().detach().numpy()
# #b11 = np.transpose(b11)
# b11 = np.reshape(b11,(3,1))
# print(np.shape(b11))
# print(type(x1))
# print(type(w11))

# a11 = np.matmul(w11,x1) + b11
# a11 = np.tanh(a11)
# print(a11)
# #print(torch.tensordot(w11,x1))


**<h2>Training the Neural Network**

In [ ]:
def training(learning_rate, nepochs,net,dataloader,test_set,test_labels):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), learning_rate) 
    #torch.optim.LBFGS(net.parameters(), lr=0.001, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, line_search_fn=None)
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.1)

    train_loss = np.zeros(nepochs)
    test_loss = np.zeros(nepochs)

    train_acc = np.zeros(nepochs)
    test_acc = np.zeros(nepochs)

    #===========================================================================
    for epoch in range(nepochs):          # loop over the dataset multiple times
    #===========================================================================
    
        running_loss = 0.0                #  Initialise losses at the start of each epoch 
        epoch_train_loss = 0.0             
        epoch_test_loss = 0.0
    
    
        counter = 0                               # ranges from 0 to the number of elements in each batch 
                                                  # eg if we have 900 train. ex. and 25 batches, there will
                                                  # be 36 elements in each batch.
        #---------------------------------------
        for i, data in enumerate(dataloader, 0):  # scan the whole dataset in each epoch, batch by batch (i ranges over batches)
        #---------------------------------------
            inputs, labels = data                 # get the inputs; data is a list of [inputs, labels]

            # zero the parameter gradients
            optimizer.zero_grad()                 # Calling .backward() mutiple times accumulates the gradient (by addition) 
                                                  # for each parameter. This is why you should call optimizer.zero_grad() 
                                                  # after each .step() call. 

            # forward + backward + optimize
        
        
            outputs = torch.zeros(np.shape(inputs)[0])
            for j in range(np.shape(inputs)[0]):
                outputs[j] = net(inputs[j][0],inputs[j][1],inputs[j][2])  # The net is designed to take a (3 x num_features)
                # tensor as an input, so when we are doing batch gd, we do a loop over all elements of the batch to create an
                # output vector with as many elements as the batch size. If our net was designed to take one row as input we 
                # wouldn't have needed the for loop, we could have had a vectorised implementation, i.e. outputs = net(inputs)
            
            
            
            loss = criterion(outputs, labels) # a single value, same as loss.item(), which is the mean loss for each mini-batch
            loss.backward()                   # performs one back-propagation step 
            optimizer.step()                  # update the network parameters (perform an update step)

            # print statistics
            running_loss += loss.item()        # loss.item() contains loss of entire mini-batch, divided by the batch size, i.e. mean loss per training example
                                               # we accumulate this loss over as many mini-batches as we like until we set it to zero after printing it
            epoch_train_loss += loss.item()    # cumulative loss for each epoch (sum of mean loss for all mini-batches)
                                               # so we ve divided here by the number of train. ex. in one mini-batch (mean)
                                               # thus all we need to do at the end of the epoch is divide by the number of mini-batches
        
            net_test_set = torch.zeros(np.shape(test_set)[0]) # outputs(predictions) of network if we input the test set
            with torch.no_grad():                             # The wrapper with torch.no_grad() temporarily sets all of 
                                                              # the requires_grad flags to false, i.e. makes all the 
                                                              # operations in the block have no gradients
                for k in range(np.shape(test_set)[0]):
                       net_test_set[k] = net(test_set[k][0],test_set[k][1],test_set[k][2])
                epoch_test_loss += criterion(net_test_set, test_labels).item() # sum test mean batch losses throughout epoch           
            if i % 10 == 2:    # print average loss every 10 mini-batches
                print('[%d, %5d] loss: %.5f' %
                      (epoch + 1, i + 1, running_loss/10))
                running_loss = 0.0
            counter += 1
            #------------------------------------       
        # Now we have added up the loss (both for training and test set) over all mini batches     
        train_loss[epoch] = epoch_train_loss/counter   # divide by number or training examples in one batch 
                                                       # to obtain average training loss for each epoch
        test_loss[epoch] = epoch_test_loss/counter
        epoch_train_loss = 0.0
        epoch_test_loss = 0.0

    #=================================================================================
    
    print('Finished Training')
    
    return train_loss, test_loss

In [ ]:
def run_for_different_num_feat(num_feat_H2O,nepochs,coordinates_water,energies_water,atomic_numbers_water,use_atom_num_as_feat):
    N_H2O           = 3            # number of atoms per molecule
    learning_rate   = 0.005#0.0001 #0.005
    batch_size      = 25#300    # 25

    data_size_H2O   = np.shape(energies_water)[0]
    test_set_size   = 100
    
    data_water = create_dataset(N_H2O, number_of_features_H2O ,coordinates_water,energies_water, \
                      atomic_numbers_water,use_atom_num_as_feat, batch_size)

    
    training_set = data_water[0];     test_set     = data_water[1];
    train_labels = data_water[2];     test_labels  = data_water[3];
    dataloader   = data_water[4];     var_lab = data_water[5]; 
    mean_lab = data_water[6];         test_set_rot = data_water[7];
    labels_norm = data_water[8];      training_set_rot = data_water[9]
        
    # Create and initialise the model
    net = BPNN_H2O(number_of_features_H2O)   #+1) # +1 if you are adding atomic number as a feature

    # Train for nepochs using learning_rate = 0.0001

    print(type(dataloader))
    losses = training(learning_rate , nepochs, net, dataloader,test_set,test_labels)
    train_loss_G_feat = losses[0]
    test_loss_G_feat  = losses[1]
    
    return train_loss_G_feat, test_loss_G_feat, test_set, net, train_labels,mean_lab, test_labels, var_lab, test_set_rot

In [ ]:
# # Training for 4 Behler and Parinello features, batch size = 300 
number_of_features_H2O   = 4       # number of features (radial and angular symmetry functions) to be used
use_atom_num_as_feat     = int(0)       # Use atomic number of each element as an extra feature for training?
nepochs                  = 120

In [ ]:
# losses_G4 =  run_for_different_num_feat(4,nepochs,coordinates_water,energies_water,atomic_numbers_water,use_atom_num_as_feat)

# train_loss_G_4_feat = losses_G4[0]
# test_loss_G_4_feat  = losses_G4[1]
# test_set            = losses_G4[2]
# net                 = losses_G4[3]
# train_labels        = losses_G4[4]
# mean_lab            = losses_G4[5]
# test_labels         = losses_G4[6]
# var_lab             = losses_G4[7]
# test_set_rot        = losses_G4[8]

In [ ]:
# Plotting loss functions
def plot_train_test_loss(train_loss, test_loss, nepochs, num_feat_G):
    x = np.arange(1,nepochs+1)

    plt.plot(x,train_loss,'blue',label = 'Training loss')
    plt.plot(x,test_loss,'red',label = 'Test loss')
#    plt.yscale('log')

    #plt.ylim([0,0.075])

    plt.ticklabel_format(useOffset=False, style='plain')
    plt.xlabel('Epoch',fontsize=14)
    plt.ylabel('Loss function',fontsize=14)
    plt.title('Average loss function per epoch for $H_2O$ training and test set',fontsize=14)
    plt.grid()
    plt.legend()
    plt.savefig('params_loss_graph_H2O_G_{0}'.format(num_feat_G),bbox_inches='tight')
    plt.show()
    
    return

In [ ]:
# plot_train_test_loss(train_loss_G_4_feat,test_loss_G_4_feat,nepochs,number_of_features_H2O)

In [ ]:
# test_set_size = 100
# prediction = np.zeros(test_set_size)
# for i in range(test_set_size):
#     x1,x2,x3 = test_set[i]
#     prediction[i] = net(x1, x2, x3)#[0]

# print(prediction)

In [ ]:
# print(np.shape(train_labels))
# train_labels = np.array(train_labels)
# print(np.mean(train_labels,axis=0))

# print(mean_lab)

In [ ]:
def plot_prediction(prediction,test_labels,var_lab,mean_lab, number_of_features_H2O):
    prediction = torch.tensor(prediction)

    x = np.linspace(min(test_labels*var_lab+mean_lab), max(test_labels*var_lab+mean_lab))
    print(min(torch.cat((test_labels,prediction),0)))
    y = x
    plt.plot(test_labels*var_lab+mean_lab,prediction*var_lab+mean_lab, 'o', color='blue', label = 'Test set')
    plt.plot(x,y, color='red',label = 'y=x')
    plt.grid()
    #plt.xlim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction),0))])
    #plt.ylim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction)))])
    #plt.ylim([-13822,-13800])
    #plt.ticklabel_format(axis="y", style="plain")
    plt.ticklabel_format(useOffset=False, style='plain')
    #plt.tick_params(axis='both',labelsize=14)
    plt.xlabel('Actual energy (kcal/mol)',fontsize=14)
    plt.ylabel('Predicted energy(kcal/mol)',fontsize=14)
    plt.title('Actual vs Predicted $H_2O$ energy',fontsize=15)
#    plt.title('Actual vs predicted $H_2O$ energy - Augmented data (10 rotations)',fontsize=15)    
    plt.legend()
    plt.savefig('predicted_energies_H2O_G_{0}'.format(number_of_features_H2O),bbox_inches='tight')
    plt.show()

    return


In [ ]:
# plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

In [ ]:
# print(net.network1.fc1.weight)

In [ ]:
# # Training for 6 Behler and Parinello features, batch size = 600 
# number_of_features_H2O   = 6       # number of features (radial and angular symmetry functions) to be used

# vars_G6 =  run_for_different_num_feat(6,nepochs,coordinates_water,energies_water,atomic_numbers_water,use_atom_num_as_feat)

# train_loss_G_6_feat = vars_G6[0]
# test_loss_G_6_feat  = vars_G6[1]
# test_set            = vars_G6[2]
# net                 = vars_G6[3]
# train_labels        = vars_G6[4]
# mean_lab            = vars_G6[5]
# test_labels         = vars_G6[6]
# var_lab             = vars_G6[7]
# test_set_rot        = vars_G6[8]

In [ ]:
# plot_train_test_loss(train_loss_G_6_feat,test_loss_G_6_feat,nepochs,number_of_features_H2O)

# test_set_size = 100

# prediction = np.zeros(test_set_size)
# for i in range(test_set_size):
#     x1,x2,x3 = test_set[i]
#     prediction[i] = net(x1, x2, x3)#[0]    
# print(prediction)

# plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

In [ ]:
# # Training for 9 Behler and Parinello features, batch size = 600 
# number_of_features_H2O   = 9       # number of features (radial and angular symmetry functions) to be used

# vars_G9 =  run_for_different_num_feat(9,nepochs,coordinates_water,energies_water,atomic_numbers_water,use_atom_num_as_feat)

# train_loss_G_9_feat = vars_G9[0]
# test_loss_G_9_feat  = vars_G9[1]
# test_set            = vars_G9[2]
# net                 = vars_G9[3]
# train_labels        = vars_G9[4]
# mean_lab            = vars_G9[5]
# test_labels         = vars_G9[6]
# var_lab             = vars_G9[7]
# test_set_rot        = vars_G9[8]
# plot_train_test_loss(train_loss_G_9_feat,test_loss_G_9_feat,nepochs,number_of_features_H2O)
# test_set_size = 100
# prediction = np.zeros(test_set_size)
# for i in range(test_set_size):
#     x1,x2,x3 = test_set[i]
#     prediction[i] = net(x1, x2, x3)#[0]    
# print(prediction)
# plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

In [ ]:
# # Training for 12 Behler and Parinello features, batch size = 600 
# number_of_features_H2O   = 12       # number of features (radial and angular symmetry functions) to be used

# vars_G12 =  run_for_different_num_feat(12,nepochs,coordinates_water,energies_water,atomic_numbers_water,use_atom_num_as_feat)

# train_loss_G_12_feat = vars_G12[0]
# test_loss_G_12_feat  = vars_G12[1]
# test_set            = vars_G12[2]
# net                 = vars_G12[3]
# train_labels        = vars_G12[4]
# mean_lab            = vars_G12[5]
# test_labels         = vars_G12[6]
# var_lab             = vars_G12[7]
# test_set_rot        = vars_G12[8]
# plot_train_test_loss(train_loss_G_12_feat,test_loss_G_12_feat,nepochs,number_of_features_H2O)
# test_set_size = 100
# prediction = np.zeros(test_set_size)
# for i in range(test_set_size):
#     x1,x2,x3 = test_set[i]
#     prediction[i] = net(x1, x2, x3)#[0]    
# print(prediction)
# plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

In [ ]:
# # Training for 16 Behler and Parinello features, batch size = 600 
# number_of_features_H2O   = 16       # number of features (radial and angular symmetry functions) to be used

# vars_G16 =  run_for_different_num_feat(16,nepochs,coordinates_water,energies_water,atomic_numbers_water,use_atom_num_as_feat)

# train_loss_G_16_feat = vars_G16[0]
# test_loss_G_16_feat  = vars_G16[1]
# test_set            = vars_G16[2]
# net                 = vars_G16[3]
# train_labels        = vars_G16[4]
# mean_lab            = vars_G16[5]
# test_labels         = vars_G16[6]
# var_lab             = vars_G16[7]
#test_set_rot         = vars_G16[8]

# plot_train_test_loss(train_loss_G_16_feat,test_loss_G_16_feat,nepochs,number_of_features_H2O)
# test_set_size = 100
# prediction = np.zeros(test_set_size)
# for i in range(test_set_size):
#     x1,x2,x3 = test_set[i]
#     prediction[i] = net(x1, x2, x3)#[0]    
# print(prediction)
# plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

In [ ]:
# #Comparing convergence plots for different numbers of features
# x = np.arange(1,nepochs+1)

# plt.plot(x[:2000],train_loss_G_4_feat[:2000],'blue',label = '4 features')
# #plt.plot(x[:2000],train_loss_G_6_feat[:2000],'red',label = '6 features')
# plt.plot(x[:2000],train_loss_G_9_feat[:2000],'green',label = '9 features')
# #plt.plot(x[:2000],train_loss_G_12_feat[:2000],'orange',label = '12 features')
# plt.plot(x[:2000],train_loss_G_16_feat[:2000],'cyan',label = '16 features')


# #plt.ylim([0,0.075])

# plt.ticklabel_format(useOffset=False, style='plain')
# plt.xlabel('Epoch',fontsize=14)
# plt.ylabel('Training loss function',fontsize=14)
# plt.title('Convergence plot for different numbers of features',fontsize=14)
# plt.grid()
# plt.legend()
# plt.savefig('loss_H2O_compar_diff_num_feat_BP_symm_random_feat',bbox_inches='tight')
# plt.show()
    

In [ ]:
# Training for 6 Behler and Parinello features, batch size = 600 
number_of_features_H2O   = 6       # number of features (radial and angular symmetry functions) to be used

vars_G6 =  run_for_different_num_feat(6,nepochs,coordinates_water,energies_water,atomic_numbers_water,use_atom_num_as_feat)

train_loss_G_6_feat = vars_G6[0]
test_loss_G_6_feat  = vars_G6[1]
test_set            = vars_G6[2]
net                 = vars_G6[3]
train_labels        = vars_G6[4]
mean_lab            = vars_G6[5]
test_labels         = vars_G6[6]
var_lab             = vars_G6[7]
test_set_rot        = vars_G6[8]

plot_train_test_loss(train_loss_G_6_feat,test_loss_G_6_feat,nepochs,number_of_features_H2O)
test_set_size = 100
prediction = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = test_set[i]
    prediction[i] = net(x1, x2, x3)#[0]    
print(prediction)
plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

**<h2>Rotating test set molecules and checking performance**

In [ ]:
print(np.shape(test_set_rot))

In [ ]:
prediction_rotated = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = test_set_rot[i]
    prediction_rotated[i] = net(x1, x2, x3)#[0]


prediction_rotated = torch.tensor(prediction_rotated)

x = np.linspace(min(test_labels*var_lab+mean_lab), max(test_labels*var_lab+mean_lab))
print(min(torch.cat((test_labels,prediction_rotated),0)))
y = x
plt.plot(test_labels*var_lab+mean_lab,prediction*var_lab+mean_lab, 'o', color='blue', label = 'Test set',markersize=8)
plt.plot(test_labels*var_lab+mean_lab,prediction_rotated*var_lab+mean_lab, '*', color='yellow', label = 'Rotated test set')

plt.plot(x,y, color='red',label = 'y=x')
plt.grid()
#plt.xlim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction),0))])
#plt.ylim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction)))])
#plt.ylim([-13822,-13800])
#plt.ticklabel_format(axis="y", style="plain")
plt.ticklabel_format(useOffset=False, style='plain')
#plt.tick_params(axis='both',labelsize=14)
plt.xlabel('Actual $H_2O$ energy value (kcal/mol)',fontsize=14)
plt.ylabel('Predicted energy value (kcal/mol)',fontsize=14)
plt.title('Actual vs Predicted energy value for rotated $H_2O$ molecules',fontsize=15)
plt.legend()
plt.savefig('rotated_predicted_energies_H2O_G_feat',bbox_inches='tight')
plt.show()


**<h1>Training on xyz coordinates**

In [ ]:
N                        = 3         # number of atoms per molecule
number_of_features_xyz   = 3         # number of features for each atom
data_size_H2O            = np.shape(energies_water)[0]                                  
training_set_size    = data_size_H2O - test_set_size


for i in range(np.shape(energies_water)[0]):  # Moving all oxygens to the origin!!!!!
    coord = coordinates_water[N*i:N*(i+1),:]
    coord = coord - coord[2,:]                
    coordinates_water[N*i:N*(i+1),:] = coord

    
# Computing variance and mean on the training data only!
coord_train = coordinates_water[:training_set_size,:]
var_train_xyz  = np.var(coord_train,axis=0)
mean_train_xyz = np.mean(coord_train,axis=0)


coordinates_water_norm = np.zeros((len(coordinates_water), number_of_features_xyz))
# normalize all data (training and test), using training set mean and variance
for i in range(np.shape(coordinates_water)[0]):
    for j in range(1,np.shape(coordinates_water)[1]):  # omit first column since for our dataset x=0 always
        coordinates_water_norm[i,j] = (coordinates_water[i,j]-mean_train_xyz[j])/var_train_xyz[j]

data_set_xyz = np.vsplit(coordinates_water_norm,data_size_H2O)     # !!!!!!!!!!!!  change to coordinates_water_norm if you are normalising
data_set_xyz = torch.FloatTensor(data_set_xyz)          # Going from a (1000,3,2) list to a a (1000,3,2) tensor


# labels same as before   
train_labels         = labels_norm[:training_set_size]
train_labels         = torch.FloatTensor(train_labels)
test_labels          = labels_norm[training_set_size:]
test_labels          = torch.FloatTensor(test_labels)

    
# Splitting the dataset into training and test set
training_set_xyz         = data_set_xyz[:training_set_size]
test_set_xyz             = data_set_xyz[training_set_size:]
#train and test labels same as before

#################################################################################
#Randomly rotating each molecule in the dataset around its oxygen 
rotated_training_set_xyz= np.zeros((np.shape(training_set_xyz)))
for i in range(training_set_size):
    coord = training_set_xyz[N*i:N*(i+1),:]
    coord = np.transpose(coord)
    A = random_rotation_matrix()
    rotated_training_set_xyz[N*i:N*(i+1),:] = np.transpose(rotate_data(A,coord))
print(np.shape(rotated_training_set_xyz))
rotated_training_set_xyz = torch.FloatTensor(rotated_training_set_xyz)
training_set_xyz = rotated_training_set_xyz
##################################################################################


#Dataset
dataset_xyz = TensorDataset(training_set_xyz, train_labels)

# Creating the batches
dataloader_xyz = torch.utils.data.DataLoader(dataset_xyz, batch_size=300, #300,
                                           shuffle=False, num_workers=2, drop_last=False) 


**<h2>Training new neural net on xyz coordinates**

In [ ]:
net_xyz = BPNN_H2O(3)    # 3 features (x,y and z)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net_xyz.parameters(), lr=0.005) #0.0001)#0.005)
#torch.optim.LBFGS(net.parameters(), lr=0.001, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, line_search_fn=None)
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.1)
nepochs = 20000

train_loss_xyz = np.zeros(nepochs)
test_loss_xyz = np.zeros(nepochs)

train_acc_xyz = np.zeros(nepochs)
test_acc_xyz = np.zeros(nepochs)

#===========================================================================
for epoch in range(nepochs):          # loop over the dataset multiple times
#===========================================================================
    
    running_loss_xyz = 0.0                #  Initialise losses at the start of each epoch 
    epoch_train_loss_xyz = 0.0             
    epoch_test_loss_xyz = 0.0
    
    
    counter = 0                               # ranges from 0 to the number of elements in each batch 
                                              # eg if we have 900 train. ex. and 25 batches, there will
                                              # be 36 elements in each batch.
    #---------------------------------------
    for i, data in enumerate(dataloader_xyz, 0):  # scan the whole dataset in each epoch, batch by batch (i ranges over batches)
    #---------------------------------------
        inputs, labels = data                 # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        optimizer.zero_grad()                 # Calling .backward() mutiple times accumulates the gradient (by addition) 
                                              # for each parameter. This is why you should call optimizer.zero_grad() 
                                              # after each .step() call. 

        # forward + backward + optimize
        
        
        outputs = torch.zeros(np.shape(inputs)[0])
        for j in range(np.shape(inputs)[0]):
            outputs[j] = net_xyz(inputs[j][0],inputs[j][1],inputs[j][2])  # The net is designed to take a (3 x num_features)
            # tensor as an input, so when we are doing batch gd, we do a loop over all elements of the batch to create an
            # output vector with as many elements as the batch size. If our net was designed to take one row as input we 
            # wouldn't have needed the for loop, we could have had a vectorised implementation, i.e. outputs = net(inputs)
                       
            
        loss = criterion(outputs, labels) # a single value, same as loss.item(), which is the mean loss for each mini-batch
        loss.backward()                   # performs one back-propagation step 
        optimizer.step()                  # update the network parameters (perform an update step)

        # print statistics
        running_loss_xyz += loss.item()        # loss.item() contains loss of entire mini-batch, divided by the batch size, i.e. mean
                                           # we accumulate this loss over as many mini-batches as we like until we set it to zero after printing it
        epoch_train_loss_xyz += loss.item()    # cumulative loss for each epoch (sum of mean loss for all mini-batches)
                                           # so we ve divided here by the number of train. ex. in one mini-batch (mean)
                                           # thus all we need to do at the end of the epoch is divide by the number of mini-batches
        
        net_test_set_xyz = torch.zeros(np.shape(test_set_xyz)[0]) # outputs(predictions) of network if we input the test set
        with torch.no_grad():                             # The wrapper with torch.no_grad() temporarily sets all of 
                                                          # the requires_grad flags to false, i.e. makes all the 
                                                          # operations in the block have no gradients
            for k in range(np.shape(test_set_xyz)[0]):
                    net_test_set_xyz[k] = net_xyz(test_set_xyz[k][0],test_set_xyz[k][1],test_set_xyz[k][2])
            epoch_test_loss_xyz += criterion(net_test_set_xyz, test_labels).item() # sum test mean batch losses throughout epoch          
        if i % 10 == 0:    # print average loss every 10 mini-batches
            print('[%d, %5d] loss: %.10f' %
                  (epoch + 1, i + 1, running_loss_xyz/10))
            running_loss_xyz = 0.0
        counter += 1
        #------------------------------------       
    # Now we have added up the loss (both for training and test set) over all mini batches     
    train_loss_xyz[epoch] = epoch_train_loss_xyz/counter   # divide by number or training examples in one batch 
                                                           # to obtain average training loss for each epoch
#     if (abs((test_loss_xyz[epoch] - test_loss_xyz[epoch - 1])/test_loss_xyz[epoch - 1])< 0.1):
#         break
    test_loss_xyz[epoch] = epoch_test_loss_xyz/counter
    epoch_train_loss_xyz = 0.0
    epoch_test_loss_xyz = 0.0

#=================================================================================
    
print('Finished Training')

In [ ]:
x = np.arange(1,nepochs+1)
# plt.plot(x[200:600],train_loss_xyz[200:600],'blue',label = 'Training loss')
# plt.plot(x[200:600],test_loss_xyz[200:600],'red',label = 'Test loss')

plt.plot(x[:epoch],train_loss_xyz[:epoch],'blue',label = 'Training loss')
plt.plot(x[:epoch],test_loss_xyz[:epoch],'red',label = 'Test loss')

#plt.ylim([-13822,-13800])

plt.ticklabel_format(useOffset=False, style='plain')
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('Loss function',fontsize=14)
plt.title('Average loss function per epoch (training with (x,y,z))',fontsize=14)
plt.grid()
plt.legend()
#plt.savefig('x_y_z_loss_graph_H2O',bbox_inches='tight')
plt.savefig('rot_training_x_y_z_loss_graph_H2O',bbox_inches='tight')
plt.show()

In [ ]:
# x = np.arange(1,nepochs+1)
# # plt.plot(x[200:600],train_loss_xyz[200:600],'blue',label = 'Training loss')
# # plt.plot(x[200:600],test_loss_xyz[200:600],'red',label = 'Test loss')

# plt.plot(x[:1500],train_loss_xyz[:1500],'green',label = 'xyz features')
# plt.plot(x[:1500],train_loss_G_3_feat[:1500],'y',label = '3 Symmetry functions')

# #plt.ylim([-13822,-13800])

# plt.ticklabel_format(useOffset=False, style='plain')
# plt.xlabel('Epoch',fontsize=14)
# plt.ylabel('Loss function',fontsize=14)
# plt.title('Convergence: Training with (x,y,z) vs with 3 symmetry functions)',fontsize=14)
# plt.grid()
# plt.legend()
# plt.savefig('x_y_z_loss_graph_H2O_compared_with_3G',bbox_inches='tight')
# plt.show()

In [ ]:
prediction_xyz = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = test_set_xyz[i]
    prediction_xyz[i] = net_xyz(x1, x2, x3)#[0]


In [ ]:
prediction_xyz = torch.tensor(prediction_xyz)

x = np.linspace(min(test_labels*var_lab+mean_lab), max(test_labels*var_lab+mean_lab))
print(min(torch.cat((test_labels,prediction_xyz),0)))
y = x
plt.plot(test_labels*var_lab+mean_lab,prediction_xyz*var_lab+mean_lab, 'o', color='blue', label = 'Test set')
plt.plot(x,y, color='red',label = 'y=x')
plt.grid()
#plt.xlim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction),0))])
#plt.ylim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction)))])
#plt.ylim([-13822,-13800])
#plt.ticklabel_format(axis="y", style="plain")
plt.ticklabel_format(useOffset=False, style='plain')
#plt.tick_params(axis='both',labelsize=14)
plt.xlabel('Actual $H_2O$ energy (kcal/mol)',fontsize=14)
plt.ylabel('Predicted energy(kcal/mol)',fontsize=14)
plt.title('Energy prediction for $H_2O$ molecules, using (x,y,z) as features',fontsize=15)
plt.legend()
#plt.savefig('xyz_predicted_energies_H2O',bbox_inches='tight')
plt.savefig('rot_train_xyz_predicted_energies_H2O',bbox_inches='tight')
plt.show()

In [ ]:
type(prediction_xyz)

In [ ]:
rotated_test_set_xyz= np.zeros((np.shape(test_set_xyz)))
for i in range(test_set_size):
    coord = test_set_xyz[N*i:N*(i+1),:]
    coord = np.transpose(coord)
    A = random_rotation_matrix()
    rotated_test_set_xyz[N*i:N*(i+1),:] = np.transpose(rotate_data(A,coord))
print(np.shape(rotated_test_set_xyz))
rotated_test_set_xyz = torch.FloatTensor(rotated_test_set_xyz)

In [ ]:
prediction_xyz_rot = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = rotated_test_set_xyz[i]
    prediction_xyz_rot[i] = net_xyz(x1, x2, x3)#[0]
print(prediction_xyz*var_lab+mean_lab)

In [ ]:
prediction_xyz_rot = torch.tensor(prediction_xyz_rot)
prediction_xyz = torch.tensor(prediction_xyz)

x = np.linspace(min(test_labels*var_lab+mean_lab), max(test_labels*var_lab+mean_lab))
print(min(torch.cat((test_labels,prediction_xyz_rot),0)))
y = x
plt.plot(test_labels*var_lab+mean_lab,prediction_xyz*var_lab+mean_lab, 'o', color='blue', label = 'Test set')
plt.plot(test_labels*var_lab+mean_lab,prediction_xyz_rot*var_lab+mean_lab, '*', color='orange', label = 'Rotated test set')
plt.plot(x,y, color='red',label = 'y=x')
plt.grid()
#plt.xlim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction),0))])
#plt.ylim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction)))])
#plt.ylim([-13822,-13800])
#plt.ticklabel_format(axis="y", style="plain")
plt.ticklabel_format(useOffset=False, style='plain')
#plt.tick_params(axis='both',labelsize=14)
plt.xlabel('Actual $H_2O$ energy(kcal/mol)',fontsize=14)
plt.ylabel('Predicted energy(kcal/mol)',fontsize=14)
plt.title('Energy prediction for $H_2O$ molecules, with (x,y,z) as features',fontsize=15)
plt.legend()
#plt.savefig('rot_xyz_predicted_energies_H2O',bbox_inches='tight')
plt.savefig('rot_rot_xyz_predicted_energies_H2O',bbox_inches='tight')
plt.show()

**<h1> Augmenting the dataset and training on xyz coordinates**

In [ ]:
N                        = 3         # number of atoms per molecule
number_of_features_xyz   = 3         # number of features for each atom
data_size_H2O            = np.shape(energies_water)[0]  
print('data_size_H2O = ', data_size_H2O)
number_of_rotations = 101

coordinates_water = (xyz_file[:,1:-1])
shape_coord_h2o = np.shape(coordinates_water)
coordinates_water = coordinates_water.astype(np.float)
new_labels        = labels_norm #torch.zeros((data_size_H2O*(number_of_rotations+1)))###### labels_norm

    
##########################################################################################################################################
# for n in range(number_of_rotations):    
#     #Randomly rotate each molecule in the data
# #    print(np.shape(coordinates_water))
#     rotated_molec_coord = np.zeros(shape_coord_h2o)
#     for i in range(data_size_H2O):
#         coord = coordinates_water[N*i:N*(i+1),:]
#         coord = coord - coord[2,:]                   # move oxygen to origin for each molecule before rotating
#         coord = np.transpose(coord)
#         A = random_rotation_matrix()
#         rotated_molec_coord[N*i:N*(i+1),:] = np.transpose(rotate_data(A,coord))
#     coordinates_water = np.vstack((coordinates_water,rotated_molec_coord))  # randomly rotate trainig and test set and compute features for them as well for later use
#     new_labels         = torch.cat((new_labels,labels_norm))   
##########################################################################################################################################

final_coords = np.zeros(((number_of_rotations)*len(coordinates_water),3))

for j in range(number_of_rotations):
    #Randomly rotate each molecule in the data
    rotated_molec_coord = np.zeros((np.shape(coordinates_water)))
    for i in range(data_size_H2O):
        coord = coordinates_water[N*i:N*(i+1),:]
        coord = coord - coord[2,:]                   # move oxygen to origin for each molecule before rotating
        coord = np.transpose(coord)
        A = random_rotation_matrix()
        rotated_molec_coord[N*i:N*(i+1),:] = np.transpose(rotate_data(A,coord))
    final_coords[len(coordinates_water)*j:len(coordinates_water)*(j+1),:] = rotated_molec_coord
#    print(rotated_molec_coord)

for j in range(number_of_rotations-1):
    new_labels     = torch.cat((new_labels,labels_norm))

print(np.shape(new_labels))
print(np.shape(final_coords))
##########################################################################################################################################
# for i in range(np.shape(energies_water)[0]):  # Moving all oxygens to the origin!!!!!
#     coord = coordinates_water[N*i:N*(i+1),:]
#     coord = coord - coord[2,:]                
#     coordinates_water[N*i:N*(i+1),:] = coord
# final_coords = np.zeros(((number_of_rotations+1)*len(coordinates_water),3))
#     #Randomly rotate each molecule in the data
# #    print(np.shape(coordinates_water))
# for i in range(data_size_H2O):
#     coord = coordinates_water[N*i:N*(i+1),:]
#     coord = coord - coord[2,:]                   # move oxygen to origin for each molecule before rotating
#     final_coords[i*(number_of_rotations+1)*N:i*(number_of_rotations+1)*N+N,:] = coord

#     coord = np.transpose(coord)
#     for n in range(number_of_rotations):    
#         A = random_rotation_matrix()
#         final_coords[i*(number_of_rotations+1)*N+(n+1)*N:i*(number_of_rotations+1)*N+(n+1)*N+N,:] = np.transpose(rotate_data(A,coord))
#         print(i*(number_of_rotations+1)*N+(n+1)*N+N)
#     new_labels[i*(number_of_rotations+1):(i+1)*(number_of_rotations+1)] = labels_norm[i]

# # print(np.shape(final_coords))        
# # print(np.shape(new_labels))        
# #for n in range(number_of_rotations):    
# #    new_labels         = torch.cat((new_labels,labels_norm))  
##########################################################################################################################################

#new_labels = torch.reshape(new_labels, (5000, 1))    


new_data_size            = len(new_labels)
test_set_size            = 2000
training_set_size        = new_data_size - test_set_size

# Computing standard deviation and mean on the training data only!
coord_train = final_coords[:training_set_size,:]          ###########
std_train_xyz  = np.std(coord_train,axis=0)
mean_train_xyz = np.mean(coord_train,axis=0)


coordinates_water_norm = np.zeros((len(final_coords), number_of_features_xyz)) ###
# normalize all data (training and test), using training set mean and standard deviation
for i in range(np.shape(final_coords)[0]):###
    for j in range(1,np.shape(final_coords)[1]):  # omit first column since for our dataset x=0 always######
        coordinates_water_norm[i,j] = (final_coords[i,j]-mean_train_xyz[j])/std_train_xyz[j]####


print(new_data_size)
data_set_xyz = np.vsplit(final_coords,new_data_size)     # !!!!!!!!!!!!  change to coordinates_water_norm if you are normalising
# print(np.shape(data_set_xyz))
data_set_xyz = torch.FloatTensor(data_set_xyz)          # Going from a (1000,3,2) list to a a (1000,3,2) tensor


# #print(np.shape(labels))
# print(np.shape(data_set_xyz))
shuffler = np.random.permutation(len(new_labels))

data_set_xyz = data_set_xyz[shuffler]

new_labels = new_labels[shuffler]

# #print(np.shape(labels))
# #print(np.shape(data_set))


# labels same as before   
train_labels         = new_labels[:training_set_size]
train_labels         = torch.FloatTensor(train_labels)
test_labels          = new_labels[training_set_size:]
test_labels          = torch.FloatTensor(test_labels)

    
# Splitting the dataset into training and test set
training_set_xyz         = data_set_xyz[:training_set_size]
test_set_xyz             = data_set_xyz[training_set_size:]
#train and test labels same as before

#Dataset
dataset_xyz = TensorDataset(training_set_xyz, train_labels)

# Creating the batches
dataloader_xyz = torch.utils.data.DataLoader(dataset_xyz, batch_size=128*10,#1000, #300,
                                           shuffle=False, num_workers=2, drop_last=False) 


In [ ]:
print(np.shape(final_coords))
print(np.shape(new_labels))

In [ ]:
# print(training_set_size)

In [ ]:
print(test_labels[0:62])

In [ ]:
net_xyz = BPNN_H2O(3)   #+1) # +1 if you are adding atomic number as a feature

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net_xyz.parameters(), lr=0.01)#0.01) #0.0001)#0.005)
#torch.optim.LBFGS(net.parameters(), lr=0.001, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, line_search_fn=None)
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.1)
nepochs = 1000

train_loss_xyz = np.zeros(nepochs)
test_loss_xyz = np.zeros(nepochs)

train_acc_xyz = np.zeros(nepochs)
test_acc_xyz = np.zeros(nepochs)

#===========================================================================
for epoch in range(nepochs):          # loop over the dataset multiple times
#===========================================================================
    
    running_loss_xyz = 0.0                #  Initialise losses at the start of each epoch 
    epoch_train_loss_xyz = 0.0             
    epoch_test_loss_xyz = 0.0
    
    
    counter = 0                               # ranges from 0 to the number of elements in each batch 
                                              # eg if we have 900 train. ex. and 25 batches, there will
                                              # be 36 elements in each batch.
    #---------------------------------------
    for i, data in enumerate(dataloader_xyz, 0):  # scan the whole dataset in each epoch, batch by batch (i ranges over batches)
    #---------------------------------------
        inputs, labels = data                 # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        optimizer.zero_grad()                 # Calling .backward() mutiple times accumulates the gradient (by addition) 
                                              # for each parameter. This is why you should call optimizer.zero_grad() 
                                              # after each .step() call. 

        # forward + backward + optimize
        
        
        outputs = torch.zeros(np.shape(inputs)[0])
        for j in range(np.shape(inputs)[0]):
            outputs[j] = net_xyz(inputs[j][0],inputs[j][1],inputs[j][2])  # The net is designed to take a (3 x num_features)
            # tensor as an input, so when we are doing batch gd, we do a loop over all elements of the batch to create an
            # output vector with as many elements as the batch size. If our net was designed to take one row as input we 
            # wouldn't have needed the for loop, we could have had a vectorised implementation, i.e. outputs = net(inputs)
                       
            
        loss = criterion(outputs, labels) # a single value, same as loss.item(), which is the mean loss for each mini-batch
        loss.backward()                   # performs one back-propagation step 
        optimizer.step()                  # update the network parameters (perform an update step)

        # print statistics
        running_loss_xyz += loss.item()        # loss.item() contains loss of entire mini-batch, divided by the batch size, i.e. mean
                                           # we accumulate this loss over as many mini-batches as we like until we set it to zero after printing it
        epoch_train_loss_xyz += loss.item()    # cumulative loss for each epoch (sum of mean loss for all mini-batches)
                                           # so we ve divided here by the number of train. ex. in one mini-batch (mean)
                                           # thus all we need to do at the end of the epoch is divide by the number of mini-batches
        
        net_test_set_xyz = torch.zeros(np.shape(test_set_xyz)[0]) # outputs(predictions) of network if we input the test set
        with torch.no_grad():                             # The wrapper with torch.no_grad() temporarily sets all of 
                                                          # the requires_grad flags to false, i.e. makes all the 
                                                          # operations in the block have no gradients
            for k in range(np.shape(test_set_xyz)[0]):
                    net_test_set_xyz[k] = net_xyz(test_set_xyz[k][0],test_set_xyz[k][1],test_set_xyz[k][2])
            epoch_test_loss_xyz += criterion(net_test_set_xyz, test_labels).item() # sum test mean batch losses throughout epoch          
        if i % 10 == 0:    # print average loss every 10 mini-batches
            print('[%d, %5d] loss: %.10f' %
                  (epoch + 1, i + 1, running_loss_xyz/10))
            running_loss_xyz = 0.0
        counter += 1
        
        
    prediction = np.zeros(test_set_size)
    for i in range(test_set_size):
        x1,x2,x3 = test_set_xyz[i]
        prediction[i] = net_xyz(x1, x2, x3)#[0]    
    plot_prediction(prediction,test_labels,var_lab,mean_lab,3)
        
        
#----------------------------------------------------------------------------------------------------------------       
    # Now we have added up the loss (both for training and test set) over all mini batches     
    train_loss_xyz[epoch] = epoch_train_loss_xyz/counter   # divide by number or training examples in one batch 
                                                           # to obtain average training loss for each epoch
#     if (abs((test_loss_xyz[epoch] - test_loss_xyz[epoch - 1])/test_loss_xyz[epoch - 1])< 0.1):
#         break
    test_loss_xyz[epoch] = epoch_test_loss_xyz/counter
    epoch_train_loss_xyz = 0.0
    epoch_test_loss_xyz = 0.0

#===============================================================================================================
    
print('Finished Training')   

In [ ]:
# train_11 = np.hstack((tr1,train_loss_xyz))#[10:]))
# test_11 = np.hstack((te1,test_loss_xyz))#[10:]))

In [ ]:
# train_101 = train_loss_xyz
# test_101 = test_loss_xyz

In [ ]:
x = np.arange(1,len(train_loss_xyz)+1)
# plt.plot(x,train_loss_xyz,'blue',label = 'Training loss')
# plt.plot(x,test_loss_xyz,'red',label = 'Test loss')

plt.plot(x[:],train_101[:],'green',label = 'train')
plt.plot(x[:],test_101[:],'red',label = 'test')
plt.yscale('log')

#plt.ylim([-13822,-13800])

#plt.ticklabel_format(useOffset=False, style='plain')
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('Loss function',fontsize=14)
plt.title('Augmented dataset convergence (100 extra rotations)',fontsize=14)
plt.grid()
plt.legend()
plt.savefig('augmented_convergence_101',bbox_inches='tight')
plt.show()

In [ ]:
# tr1 = train_loss_xyz
# te1 = test_loss_xyz

In [ ]:
#plot_train_test_loss(train_loss_xyz_aug,test_loss_xyz_aug,nepochs,3)

test_set_size = 2000

prediction = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = test_set_xyz[i]
    prediction[i] = net_xyz(x1, x2, x3)#[0]    
#print(prediction)

rotated_test_set_aug= np.zeros((np.shape(test_set_xyz)))
for i in range(test_set_size):
    coord = test_set_xyz[N*i:N*(i+1),:]
    coord = np.transpose(coord)
    A = random_rotation_matrix()
    rotated_test_set_aug[N*i:N*(i+1),:] = np.transpose(rotate_data(A,coord))
rotated_test_set_aug = torch.FloatTensor(rotated_test_set_aug)

prediction_rot_aug = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = rotated_test_set_aug[i]
    prediction_rot_aug[i] = net_xyz(x1, x2, x3)#[0]

prediction_rot_aug = torch.tensor(prediction_rot_aug)
prediction = torch.tensor(prediction)

x = np.linspace(min(test_labels*var_lab+mean_lab), max(test_labels*var_lab+mean_lab))
print(min(torch.cat((test_labels,prediction_rot_aug),0)))
y = x
plt.plot(test_labels*var_lab+mean_lab,prediction*var_lab+mean_lab, 'o', color='blue', label = 'Test set')
#plt.plot(test_labels*var_lab+mean_lab,prediction_rot_aug*var_lab+mean_lab, '*', color='orange', label = 'Rotated test set')
plt.plot(x,y, color='red',label = 'y=x')
plt.grid()
#plt.xlim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction),0))])
#plt.ylim([min(torch.cat((test_labels,prediction),0)), max(torch.cat((test_labels,prediction)))])
#plt.ylim([-13822,-13800])
#plt.ticklabel_format(axis="y", style="plain")
plt.ticklabel_format(useOffset=False, style='plain')
#plt.tick_params(axis='both',labelsize=14)
plt.xlabel('Actual $H_2O$ energy(kcal/mol)',fontsize=14)
plt.ylabel('Predicted energy(kcal/mol)',fontsize=14)
plt.title('Energy prediction $H_2O$, augmented data (100 rotations)',fontsize=15)
plt.legend()
#plt.savefig('rot_xyz_predicted_energies_H2O',bbox_inches='tight')
plt.savefig('predicted_energies_H2O_aug_101',bbox_inches='tight')
plt.show()

In [ ]:
# #plot_train_test_loss(train_loss_xyz_aug,test_loss_xyz_aug,nepochs,3)

# test_set_size = 200
# test_set_xyz_reduced = test_set_xyz[0:200]
# test_labels_reduced = test_labels[0:200]
# prediction = np.zeros(test_set_size)
# for i in range(test_set_size):
#     x1,x2,x3 = test_set_xyz_reduced[i]
#     prediction[i] = net_xyz(x1, x2, x3)#[0]    
# #print(prediction)

# rotated_test_set_aug = np.zeros((np.shape(test_set_xyz_reduced)))
# for i in range(test_set_size):
#     coord = test_set_xyz_reduced[N*i:N*(i+1),:]
#     coord = np.transpose(coord)
#     A = random_rotation_matrix()
#     rotated_test_set_aug[N*i:N*(i+1),:] = np.transpose(rotate_data(A,coord))
# rotated_test_set_aug = torch.FloatTensor(rotated_test_set_aug)

# prediction_rot_aug = np.zeros(test_set_size)
# for i in range(test_set_size):
#     x1,x2,x3 = rotated_test_set_aug[i]
#     prediction_rot_aug[i] = net_xyz(x1, x2, x3)#[0]

# prediction_rot_aug = torch.tensor(prediction_rot_aug)
# prediction = torch.tensor(prediction)

# x = np.linspace(min(test_labels_reduced*var_lab+mean_lab), max(test_labels_reduced*var_lab+mean_lab))
# print(min(torch.cat((test_labels_reduced,prediction_rot_aug),0)))
# y = x
# plt.plot(test_labels_reduced*var_lab+mean_lab,prediction*var_lab+mean_lab, 'o', color='blue', label = 'Test set')
# plt.plot(test_labels_reduced*var_lab+mean_lab,prediction_rot_aug*var_lab+mean_lab, '*', color='orange', label = 'Rotated test set')
# plt.plot(x,y, color='red',label = 'y=x')
# plt.grid()
# #plt.xlim([min(torch.cat((test_labels_reduced,prediction),0)), max(torch.cat((test_labels_reduced,prediction),0))])
# #plt.ylim([min(torch.cat((test_labels_reduced,prediction),0)), max(torch.cat((test_labels_reduced,prediction)))])
# #plt.ylim([-13822,-13800])
# #plt.ticklabel_format(axis="y", style="plain")
# plt.ticklabel_format(useOffset=False, style='plain')
# #plt.tick_params(axis='both',labelsize=14)
# plt.xlabel('Actual $H_2O$ energy(kcal/mol)',fontsize=14)
# plt.ylabel('Predicted energy(kcal/mol)',fontsize=14)
# plt.title('Energy prediction $H_2O$, augmented data (50 rotations)',fontsize=15)
# plt.legend()
# #plt.savefig('rot_xyz_predicted_energies_H2O',bbox_inches='tight')
# plt.savefig('predicted_energies_H2O_aug_51_rot',bbox_inches='tight')
# plt.show()

**<h2> Redundant Internal Coordinates - Pairwise Distances**

In [ ]:
data_size            = np.shape(energies_water)[0]        # We have 1000 water molecule conformations
test_set_size        = 100
training_set_size    = data_size - test_set_size
    
number_of_features = 3
G = np.zeros((len(coordinates_water), number_of_features)) 

for i in range(data_size):                
    coord = coordinates_water[N*i:N*(i+1),:]
    G[N*i:N*(i+1),:]   = pairwise_distances(coord)
    
# Computing variance and mean on the training data only!
G_train = G[:training_set_size,:]
var  = np.std(G_train,axis=0)
mean = np.mean(G_train,axis=0)
print(mean)
G_norm = np.zeros((len(coordinates_water), number_of_features))
print(np.shape(G))
# normalize all data (training and test), using training set mean and variance
for i in range(np.shape(G)[0]):
    for j in range(np.shape(G)[1]):
        G_norm[i,j] = (G[i,j]-mean[j])/var[j]   
        
print(type(G_norm))  
    
if use_atom_num_as_feat == 1:
    G_norm = np.append(G_norm, atomic_numbers, axis=1)    # Adding atomic number as a feature
       
        
data_set = np.vsplit(G_norm,data_size)       # Group every three atoms that constitute a molecule together  !!!!!!! G_norm
#data_set = np.random.permutation(training_set)
data_set = torch.FloatTensor(data_set)                    # Going from a (1000,3,2) list to a a (1000,3,2) tensor
   
# print(data_set[0])
# print(data_set[0][1][1])

labels = energies_water          # turning energies_water into a (1000) tensor

    
# #     print(np.shape(labels))
# #     print(np.shape(data_set))
#     shuffler = np.random.permutation(len(labels))

#     data_set = data_set[shuffler]

#     labels = labels[shuffler]

# #     print(np.shape(labels))
# #     print(np.shape(data_set))
    
    
# Computing variance and mean on the training data only!
lab_train = labels[:training_set_size]
var_lab  = np.std(lab_train,axis=0)
mean_lab = np.mean(lab_train,axis=0)

labels_norm = np.zeros((np.shape(labels)))
# normalize all data (training and test), using training set mean and variance
for i in range(np.shape(labels)[0]):
    labels_norm[i] = (labels[i]-mean_lab)/var_lab  
    
    
labels_norm = torch.FloatTensor(labels_norm)   # !!!!!!!!!!labels_norm    
    
    
# Splitting the dataset into training and test set
training_set         = data_set[:training_set_size]
test_set             = data_set[training_set_size:data_size]
raining_set_rot     = data_set[data_size:data_size+training_set_size]
test_set_rot         = data_set[data_size+training_set_size:]

train_labels         = labels_norm[:training_set_size]
train_labels         = torch.FloatTensor(train_labels)
test_labels          = labels_norm[training_set_size:]
test_labels          = torch.FloatTensor(test_labels)

# Dataset
dataset = TensorDataset(training_set, train_labels)
#print(dataset[0])

# Creating the batches
dataloader = torch.utils.data.DataLoader(dataset, batch_size=25, #300,
                                           shuffle=True, num_workers=2, drop_last=False) # ?????

print(np.shape(training_set))
    


In [ ]:
net = BPNN_H2O(3)   #+1) # +1 if you are adding atomic number as a feature
nepochs= 120
learning_rate = 0.005
losses = training(learning_rate , nepochs, net, dataloader,test_set,test_labels)
train_loss_G_ric1_feat = losses[0]
test_loss_G_ric1_feat  = losses[1]

plot_train_test_loss(train_loss_G_ric1_feat,test_loss_G_ric1_feat,nepochs,number_of_features_H2O)
test_set_size = 100
prediction = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = test_set[i]
    prediction[i] = net(x1, x2, x3)#[0]    
print(prediction)
plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

In [ ]:
x = np.arange(1,nepochs+1)

plt.plot(x[:],train_loss_G_6_feat[:],'green',label = 'BP Symmetry functions')
plt.plot(x[:],train_loss_G_ric1_feat[:],'red',label = 'Pairwise distances')
plt.yscale('log')

#plt.ticklabel_format(useOffset=False, style='plain')
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('Training Loss function',fontsize=14)
plt.title('Convergence using BP symmetry functions vs Pairwise distances',fontsize=14)
plt.grid()
plt.legend()
plt.savefig('bp_vs_pair_dist',bbox_inches='tight')
plt.show()


**<h2> Redundant Internal Coordinates - Pairwise Distances and Angles**

In [ ]:
data_size            = np.shape(energies_water)[0]        # We have 1000 water molecule conformations
test_set_size        = 100
training_set_size    = data_size - test_set_size
    
number_of_features = 3 # 3 distances and 1 angle

G = np.zeros((len(coordinates_water), number_of_features)) 
costheta = np.zeros((len(coordinates_water),1))
for i in range(data_size):                
    coord = coordinates_water[N*i:N*(i+1),:]
    G[N*i:N*(i+1),:]      = pairwise_distances(coord)
    costhetaijk = np.zeros((3,1))
    Dp = pairwise_distances(coord)

    R_vec_1 = coord[0,:] - coord[1,:]
    R_vec_2 = coord[0,:] - coord[2,:]
    costhetaijk[0]  = np.dot(R_vec_1, R_vec_2)/(Dp[0][1]*Dp[0][2])
    
    R_vec_1 = coord[1,:] - coord[0,:]
    R_vec_2 = coord[1,:] - coord[2,:]
    costhetaijk[1]  = np.dot(R_vec_1, R_vec_2)/(Dp[1][0]*Dp[1][2])
    
    R_vec_1 = coord[2,:] - coord[0,:]
    R_vec_2 = coord[2,:] - coord[1,:]
    costhetaijk[2]  = np.dot(R_vec_1, R_vec_2)/(Dp[2][0]*Dp[2][1])
    
#    print(costhetaijk)

    costheta[N*i:N*(i+1),:]  = costhetaijk

G = np.append(G, costheta, axis=1)

# Computing variance and mean on the training data only!
G_train = G[:training_set_size,:]
var  = np.std(G_train,axis=0)
mean = np.mean(G_train,axis=0)
print(mean)
G_norm = np.zeros((len(coordinates_water), number_of_features+1))
print(np.shape(G))
# normalize all data (training and test), using training set mean and variance
for i in range(np.shape(G)[0]):
    for j in range(np.shape(G)[1]):
        G_norm[i,j] = (G[i,j]-mean[j])/var[j]   
        
print(type(G_norm))  
    
if use_atom_num_as_feat == 1:
    G_norm = np.append(G_norm, atomic_numbers, axis=1)    # Adding atomic number as a feature
       
        
data_set = np.vsplit(G_norm,data_size)       # Group every three atoms that constitute a molecule together  !!!!!!! G_norm
#data_set = np.random.permutation(training_set)
data_set = torch.FloatTensor(data_set)                    # Going from a (1000,3,2) list to a a (1000,3,2) tensor
   

labels = energies_water          # turning energies_water into a (1000) tensor    
    
# Computing variance and mean on the training data only!
lab_train = labels[:training_set_size]
var_lab  = np.std(lab_train,axis=0)
mean_lab = np.mean(lab_train,axis=0)

labels_norm = np.zeros((np.shape(labels)))
# normalize all data (training and test), using training set mean and variance
for i in range(np.shape(labels)[0]):
    labels_norm[i] = (labels[i]-mean_lab)/var_lab  
    
    
labels_norm = torch.FloatTensor(labels_norm)   # !!!!!!!!!!labels_norm    
    
    
# Splitting the dataset into training and test set
training_set         = data_set[:training_set_size]
test_set             = data_set[training_set_size:data_size]
raining_set_rot     = data_set[data_size:data_size+training_set_size]
test_set_rot         = data_set[data_size+training_set_size:]

train_labels         = labels_norm[:training_set_size]
train_labels         = torch.FloatTensor(train_labels)
test_labels          = labels_norm[training_set_size:]
test_labels          = torch.FloatTensor(test_labels)

# Dataset
dataset = TensorDataset(training_set, train_labels)
#print(dataset[0])

# Creating the batches
dataloader = torch.utils.data.DataLoader(dataset, batch_size=25, #300,
                                           shuffle=True, num_workers=2, drop_last=False) # ?????

print(np.shape(training_set))
    


In [ ]:
net = BPNN_H2O(4)   #+1) # +1 if you are adding atomic number as a feature
nepochs= 120
learning_rate = 0.005
losses = training(learning_rate , nepochs, net, dataloader,test_set,test_labels)
train_loss_G_ric2_feat = losses[0]
test_loss_G_ric2_feat  = losses[1]

plot_train_test_loss(train_loss_G_ric2_feat,test_loss_G_ric2_feat,nepochs,number_of_features_H2O)
test_set_size = 100
prediction = np.zeros(test_set_size)
for i in range(test_set_size):
    x1,x2,x3 = test_set[i]
    prediction[i] = net(x1, x2, x3)#[0]    
print(prediction)
plot_prediction(prediction,test_labels,var_lab,mean_lab,number_of_features_H2O)

In [ ]:
x = np.arange(1,nepochs+1)

plt.plot(x[:],train_loss_G_6_feat[:],'green',label = 'BP Symmetry functions')
plt.plot(x[:],train_loss_G_ric1_feat[:],'red',label = 'Pairwise distances')
plt.plot(x[:],train_loss_G_ric2_feat[:],'blue',label = 'Pairwise distances & Angles')

plt.yscale('log')

#plt.ticklabel_format(useOffset=False, style='plain')
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('Training Loss function',fontsize=14)
plt.title('Training loss plots ',fontsize=14)
plt.grid()
plt.legend()
plt.savefig('bp_vs_pair_dist',bbox_inches='tight')
plt.show()


**<h2>Atomic local frames** 

Each atom will be represented by three features - its x,y and z coordinates with respect to the origin. For each water molecule, the origin is placed on the oxygen atom. The x-axis is set to coincide with the first OH bond, while the xy plane is the plane defined by the three atoms H-O-H. The z axis is set to be perpendicular to the H-O-H (xy) plane, so that the x,y and z axes form an orthogonal coordinate system.

In [ ]:
print(np.shape(coordinates_water))
print(coordinates_water)

In [ ]:
data_size            = np.shape(energies_water)[0]        # We have 1000 water molecule conformations
test_set_size        = 100
training_set_size    = data_size - test_set_size
    
number_of_features = 3    # 3 cartesian coordinates

coord = coordinates_water[0]
print(coord)

**<h2>Predicting Hydrogen ($H_2$) Energies**

In [ ]:
energies_H2 = np.genfromtxt('./h2/energies.txt')
print("Energies file has",np.shape(energies_H2),"entries")
#geometry_data =  read('./water/structures.xyz',index=':')
#print("Geometry file has",np.shape(geometry_data),"entries")
#print(geometry_data[0])
#geometry_data = np.array(geometry_data)


In [ ]:
# see https://education.molssi.org/python-data-analysis/01-numpy-arrays/index.html
#https://stackoverflow.com/questions/23353585/got-1-columns-instead-of-error-in-numpy

file_location = os.path.join('h2', 'structures.xyz')
xyz_file = np.genfromtxt(fname=file_location, skip_header=2, dtype='unicode',invalid_raise = False)
# where invalid_raise = False was used to skip all lines in the xyz file that only have one column
symbols = xyz_file[:,0]
coordinates_H2 = (xyz_file[:,1:-1])
coordinates_H2 = coordinates_H2.astype(np.float)

#print(symbols)
#print(coordinates)

In [ ]:
atomic_numbers_H2 = (xyz_file[:,-1])

atomic_numbers_H2 = atomic_numbers_H2.astype(int)
atomic_numbers_H2 = np.reshape(atomic_numbers_H2,(len(coordinates_H2),1))
#atomic_numbers = torch.from_numpy(atomic_numbers)
print(type(atomic_numbers_H2))
#print(atomic_numbers_H2)
print(np.shape(atomic_numbers_H2))

In [ ]:
# N_H2                    = 2           # number of atoms per molecule
# number_of_features_H2   = 6           # number of features (symmetry functions) for each atom (we create one radial)
#                                    # and one angular, but can create more by vaying the parameters η, λ, ζ, Rs etc.

In [ ]:
# data_H2 = create_dataset(N_H2,number_of_features_H2,coordinates_H2,energies_H2, \
#                       atomic_numbers_H2,1, 200)

# training_set_H2 = data_H2[0]
# print('Training set:')
# print(type(training_set_H2))
# print(np.shape(training_set_H2))
# test_set_H2     = data_H2[1]
# print('\n')
# print('Test set:')
# print(type(test_set_H2))
# print(np.shape(test_set_H2))
# train_labels_H2 = data_H2[2]
# print('\n')
# print('Training labels:')
# print(type(train_labels_H2))
# print(np.shape(train_labels_H2))
# test_labels_H2  = data_H2[3]
# print('\n')
# print('Test labels:')
# print(type(test_labels_H2))
# print(np.shape(test_labels_H2))
# dataloader_H2   = data_H2[4]
# print('\n')
# print('data_H2loader:')
# print(type(dataloader_H2))
# print(np.shape(dataloader_H2))

# var_lab_H2 = data_H2[5]
# print('\n')
# print('Variance of labels:')
# print(type(var_lab_H2))
# print(var_lab_H2)

# mean_lab_H2 = data_H2[6]
# print('\n')
# print('Mean value of labels:')
# print(type(mean_lab_H2))
# print(mean_lab_H2)

# test_set_rot_H2 = data_H2[7]
# print('\n')
# print('Rotated test set:')
# print(type(test_set_rot_H2))
# print(np.shape(test_set_rot_H2))

# labels_norm_H2 = data_H2[8]
# print('\n')
# print('Normalised labels:')
# print(type(labels_norm_H2))
# print(np.shape(labels_norm_H2))

# training_set_rot = data_H2[9]


**Creating features using only radial symmetry functions**

In [ ]:
N_H2                    = 2           # number of atoms per molecule
number_of_features_H2   = 6           # number of features (symmetry functions) for each atom
                                       
    
# heta   = np.linspace(0.01, 4, num=number_of_features_H2)
# random.shuffle(heta)

# Rs     = np.linspace(0, 1, num=number_of_features_H2)
# random.shuffle(Rs)

# lambdaa = np.ones(number_of_features_H2)
# random.shuffle(lambdaa)

# zeta    = np.linspace(0, 8, num=number_of_features_H2)
# random.shuffle(zeta)


heta    = [0.3,  4.,    3.202, 1.606, 2.404, 0.808] 
zeta    = [8.,  0, 3.2, 4.8 , 6.4, 8 ]
Rs      = [0.8, 0.4, 2, 1. ,  0.,  0.6]
lambdaa = [1., 1., 1., 1. , 1., 1.]


data_size_H2            = np.shape(energies_H2)[0]        # We have 500 H2 molecule conformations
training_set_size_H2    = data_size_H2 - 50

    
G_H2 = np.zeros((len(coordinates_H2), number_of_features_H2))  # we have 3000x2 features (2 symm funcs for ech of the 3000 atoms in the dataset)

for i in range(data_size_H2):
    coord = coordinates_H2[N_H2*i:N_H2*(i+1),:]
    Dp    = pairwise_distances(coord)
    for j in range(0,number_of_features_H2):
        G_H2[N_H2*i:N_H2*(i+1),j]   = radial_BP_symm_func(Dp,N_H2,heta[j],Rs[j])     
    
# Computing variance and mean on the training data only!
G_train_H2 = G_H2[:training_set_size_H2,:]
var_train_H2  = np.var(G_train_H2,axis=0)
mean_train_H2 = np.mean(G_train_H2,axis=0)

G_norm_H2 = np.zeros((len(coordinates_H2), number_of_features_H2))
# normalize all data (training and test), using training set mean and variance
for i in range(np.shape(G_H2)[0]):
    for j in range(np.shape(G_H2)[1]):
        G_norm_H2[i,j] = (G_H2[i,j]-mean_train_H2[j])/var_train_H2[j]   

G_norm_H2 = np.append(G_norm_H2, atomic_numbers_H2, axis=1)
        
        
data_set_H2 = np.vsplit(G_norm_H2,data_size_H2)     # Going from a (3000,2) np.array to a (1000,3,2) list
#data_set = np.random.permutation(training_set)
data_set_H2 = torch.FloatTensor(data_set_H2)          # Going from a (1000,3,2) list to a a (1000,3,2) tensor

# print(data_set[0])
# print(data_set[0][1][1])

labels_H2 = energies_H2          # turning energies into a (1000) tensor


# Computing variance and mean on the training data only!
lab_train_H2 = labels_H2[:training_set_size_H2]
var_lab_H2  = np.var(lab_train_H2,axis=0)
mean_lab_H2 = np.mean(lab_train_H2,axis=0)
print(mean_lab_H2)

labels_norm_H2 = np.zeros((np.shape(labels_H2)))
# normalize all data (training and test), using training set mean and variance
for i in range(np.shape(labels_H2)[0]):
    labels_norm_H2[i] = (labels_H2[i]-mean_lab_H2)/var_lab_H2  
    
    
labels_norm_H2 = torch.FloatTensor(labels_norm_H2)      
    
    
# # Splitting the dataset into training and test set
# training_set_H2         = data_set_H2[:training_set_size_H2]
# test_set_H2             = data_set_H2[training_set_size_H2:]

# train_labels_H2         = labels_norm_H2[:training_set_size_H2]
# test_labels_H2          = labels_norm_H2[training_set_size_H2:]

# # Dataset
# dataset_H2 = TensorDataset(training_set_H2, train_labels_H2)
# #print(dataset[0])

# # Creating the batches
# dataloader_H2 = torch.utils.data.DataLoader(dataset_H2, batch_size=400,
#                                            shuffle=True, num_workers=2, drop_last=False) # ?????

# print(np.shape(training_set_H2))

In [ ]:
# print(G_norm[:100,:-1])

In [ ]:
class Subnets_h2(nn.Module):
    def __init__(self):
        super(Subnets_h2, self).__init__()
        self.fc1 = nn.Linear(7,3)        # where fc stands for fully connected 
        self.fc2 = nn.Linear(3,3)        
        self.fc3 = nn.Linear(3, 1)


    def forward(self, x):
        x = torch.tanh(self.fc1(x))           # Apply a tanh activation on fully connected layer 1 
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)                   # Using a linear function (identity function) for the subnet output layers
        return x

class BPNN_h2(nn.Module):
    def __init__(self):
        super(BPNN_h2, self).__init__()
        self.network1 = Subnets_h2()
        self.network2 = Subnets_h2()
        
#        self.fc_out = nn.Linear(3, 1)      # should this be defined here, given that we are not trying to optimise
        
    def forward(self, x1, x2):
        x1 = self.network1(x1)
        x2 = self.network2(x2)
        
#         print(x1)
#         print(x2)
        
        x = torch.cat((x1, x2), 0) 
        x = torch.sum(x)                   #??????????????????????????? try average pooling?
        x = torch.reshape(x,[1])
        return x



model_h2 = BPNN_h2()
    
    
    
print('Copying weights!')
#'Network1','layer 1'
# 'weights'
model_h2.network1.fc1.weight = net.network1.fc1.weight 
# print('biases')
model_h2.network1.fc1.bias = net.network1.fc1.bias

# print('layer 2')
# print('weights')
model_h2.network1.fc2.weight = net.network1.fc2.weight 
# print('biases')
model_h2.network1.fc2.bias = net.network1.fc2.bias

# print('layer 3')
# print('weights')
model_h2.network1.fc3.weight = net.network1.fc3.weight 
# print('biases')
model_h2.network1.fc3.bias = net.network1.fc3.bias



#'Network2','layer 1'
# 'weights'
model_h2.network2.fc1.weight = net.network2.fc1.weight 
# print('biases')
model_h2.network2.fc1.bias = net.network2.fc1.bias


# print('layer 2')
# print('weights')
model_h2.network2.fc2.weight = net.network2.fc2.weight 
# print('biases')
model_h2.network2.fc2.bias = net.network2.fc2.bias

# print('layer 3')
# print('weights')
model_h2.network2.fc3.weight = net.network2.fc3.weight 
# print('biases')
model_h2.network2.fc3.bias = net.network2.fc3.bias


print('Finished copying weights!')


In [ ]:
x1, x2 = data_set_H2[0]
print('x1',x1)
print('x2',x2)

output = model_h2(x1, x2)
print('output')
print(output)
print(output*var_lab_H2+mean_lab_H2)
print(np.shape(output))

In [ ]:
prediction_H2 = torch.zeros(data_size_H2)
for i in range(data_size_H2):
    x1, x2 = data_set_H2[i]
#     print('x1',x1)
#     print('x2',x2)
    with torch.no_grad():
        prediction_H2[i] = model_h2(x1, x2)
    print(prediction_H2[i]*var_lab_H2+mean_lab_H2)


In [ ]:
# print(prediction_H2*var_lab_H2+mean_lab_H2)

In [ ]:
print(np.shape(labels_norm_H2))
labels_norm_H2 = np.array(labels_norm_H2)
print(np.mean(labels_norm_H2*var_lab_H2+mean_lab_H2,axis=0))

In [ ]:
prediction_H2 = torch.tensor(prediction_H2)

x = np.linspace(min(labels_H2), max(labels_H2))
#print(min(torch.cat((test_labels_H2,prediction_H2),0)))
y = x
plt.plot(labels_H2,prediction_H2*var_lab_H2+mean_lab_H2, 'o', color='blue', label = 'test set')
plt.plot(x,y, color='red',label = 'y=x')
plt.grid()
#plt.xlim([min(torch.cat((test_labels,prediction_H2),0)), max(torch.cat((test_labels,prediction_H2),0))])
#plt.ylim([min(torch.cat((test_labels,prediction_H2),0)), max(torch.cat((test_labels,prediction_H2)))])
#plt.ylim([-500,-700])
#plt.ticklabel_format(axis="y", style="plain")
plt.ticklabel_format(useOffset=False, style='plain')
#plt.tick_params(axis='both',labelsize=14)
plt.xlabel('Actual $H_2$ energy value (kcal/mol)',fontsize=14)
plt.ylabel('Predicted $H_2$ energy value (kcal/mol)',fontsize=14)
plt.title('Actual vs Predicted energy value for $H_2$ molecules',fontsize=15)
plt.legend()
plt.savefig('predicted_energies_H2_using_H2O_trained_net',bbox_inches='tight')
plt.show()


In [ ]:
print('layer 1')
print('weights')
print(net.network1.fc1.weight)
print('biases')
print(net.network1.fc1.bias)

**<h3>Drawing a neural network using matplotlib**

In [ ]:
import matplotlib.pyplot as plt

def draw_neural_net(ax, left, right, bottom, top, layer_sizes):
    n_layers = len(layer_sizes)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            x = n*h_spacing + left
            y = layer_top - m*v_spacing
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                            color='w', ec='k', zorder=2.7)
            ax.add_artist(circle)
            if n == 0:
                lab   = "$G_{0}$".format(m+1)
                label = ax.annotate(lab, xy=(x, y-0.01), fontsize=20, ha="center")
            if n == 1:
                lab   = '$y^1_{0}$'.format(m+1)
                label = ax.annotate(lab, xy=(x, y-0.01), fontsize=20, ha="center")
            if n == 2:
                lab   = '$y^2_{0}$'.format(m+1)
                label = ax.annotate(lab, xy=(x, y-0.01), fontsize=20, ha="center")
            if n == 3:
                lab   = '$E_{tot}$'
                label = ax.annotate(lab, xy=(x, y-0.01), fontsize=20, ha="center")
            
            
        # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                    [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k')
                ax.add_artist(line)
    
    
    
    


In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.gca()
ax.axis('off')
draw_neural_net(ax, .1, .9, .1, .9, [1, 5, 5, 1])
fig.savefig('fc_nn_gen.png')